# Задача: создание модели нейронной сети для определения тональности комментариев

### Основные этапы реализации:

> Исследование и очистка данных

> Стандартизация данных в формат, пригодный для обработки нейросетью

> Определение структуры и слоев нейронной сети

> Обучение модели

> Тестирование модели и проверка метрик точности

> Подготовка к практическому применению модели

### Подключение первичных необходимых библиотек

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import os

In [20]:
pip install openpyxl


   -------------------- ------------------- 1/2 [openpyxl]
   -------------------- ------------------- 1/2 [openpyxl]
   -------------------- ------------------- 1/2 [openpyxl]
   ---------------------------------------- 2/2 [openpyxl]

Note: you may need to restart the kernel to use updated packages.


In [164]:
pip install pymorphy2


  Preparing metadata (setup.py): started
  Preparing metadata (setup.py): finished with status 'done'
   ---------------------------------------- 0.0/8.2 MB ? eta -:--:--
   ---------------------------------------- 0.0/8.2 MB ? eta -:--:--
   ---------------------------------------- 0.0/8.2 MB ? eta -:--:--
   ---------------------------------------- 0.0/8.2 MB ? eta -:--:--
   - -------------------------------------- 0.3/8.2 MB ? eta -:--:--
   - -------------------------------------- 0.3/8.2 MB ? eta -:--:--
   -- ------------------------------------- 0.5/8.2 MB 578.7 kB/s eta 0:00:14
   -- ------------------------------------- 0.5/8.2 MB 578.7 kB/s eta 0:00:14
   --- ------------------------------------ 0.8/8.2 MB 599.2 kB/s eta 0:00:13
   --- ------------------------------------ 0.8/8.2 MB 599.2 kB/s eta 0:00:13
   ----- ---------------------------------- 1.0/8.2 MB 613.9 kB/s eta 0:00:12
   ------ --------------------------------- 1.3/8.2 MB 627.1 kB/s eta 0:00:12
   ------ -----

### Загрузка комментариев и работа с датасетом

In [141]:
# file_14k = os.path.join(os.path.expanduser("~"), "Desktop\ML2\Toxic_comments_ai\Data\comments.xlsx")
# file_250k = os.path.join(os.path.expanduser("~"), "Desktop\ML2\Toxic_comments_ai\Data\dataset.txt")

file_14k = r"D:\YupiterProjects4course\ML2\Toxic_comments_ai\Data\comments.xlsx"
file_250k = r"D:\YupiterProjects4course\ML2\Toxic_comments_ai\Data\dataset.txt"

with open(file_14k, encoding = 'utf-8') as file:
    print("Успех")
with open(file_250k, encoding = 'utf-8') as file:
    print("Успех")

df = pd.read_excel(file_14k, skiprows=1)

Успех
Успех


In [142]:
df

,comment,toxic
0,"Верблюдов-то за что? Дебилы, бл...\n",1.0
1,"Хохлы, это отдушина затюканого россиянина, мол...",1.0
2,Собаке - собачья смерть\n,1.0
3,"Страницу обнови, дебил. Это тоже не оскорблени...",1.0
4,"тебя не убедил 6-страничный пдф в том, что Скр...",1.0
...,...,...
14407,Вонючий совковый скот прибежал и ноет. А вот и...,1.0
14408,А кого любить? Гоблина тупорылого что-ли? Или ...,1.0
14409,"Посмотрел Утомленных солнцем 2. И оказалось, ч...",0.0
14410,КРЫМОТРЕД НАРУШАЕТ ПРАВИЛА РАЗДЕЛА Т.К В НЕМ Н...,1.0


In [143]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 14412 entries, 0 to 14411
Data columns (total 2 columns):
 #   Column   Non-Null Count  Dtype  
---  ------   --------------  -----  
 0   comment  14412 non-null  object 
 1   toxic    14412 non-null  float64
dtypes: float64(1), object(1)
memory usage: 225.3+ KB


In [144]:
df['toxic'] = df['toxic'].astype(int)

In [145]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 14412 entries, 0 to 14411
Data columns (total 2 columns):
 #   Column   Non-Null Count  Dtype 
---  ------   --------------  ----- 
 0   comment  14412 non-null  object
 1   toxic    14412 non-null  int32 
dtypes: int32(1), object(1)
memory usage: 169.0+ KB


In [146]:
df.toxic.value_counts()

toxic
0    9586
1    4826
Name: count, dtype: int64

In [147]:
df.comment[14409]

'Посмотрел Утомленных солнцем 2. И оказалось, что это хороший фильм, такая высокобюджетная артхаусятина, к которой могут быть претензии только потому, что спиздили-распилили и вообще ТАК НЕ БЫВАЕТ. Ну нахуй этих критиков. Обзоры длиннее фильмов, петросянство хуже рашкокомедий, ебанутая ненависть и доебки по мелочам.\n'

In [148]:
df.comment[14411]

'До сих пор пересматриваю его видео. Орамбо кстати на своем канале пилит похожий контент, но качеством похуже, там же и Шуран не редко светится, храню хрупкую надежду что когда-то он вернется, такая годнота ведь.\n'

In [149]:
def create_df():
    data_list = []
    with open(file_250k, 'r', encoding = 'utf-8') as file:
        for idx, line in enumerate(file):
            parts = line.split(maxsplit=1)
            labels = parts[0].replace('__label__','')
            text = parts[1].strip()
            mask = [1 if "NORMAL" in labels else 0,
                    1 if "INSULT" in labels else 0,
                    1 if "THREAT" in labels else 0,
                    1 if "OBSCENITY" in labels else 0]
            data_list.append((text, *mask))
        return pd.DataFrame(data_list, columns=["comment", "normal", "insult", "threat", "obscenity"])

In [150]:
df2 = create_df()

In [151]:
df2

,comment,normal,insult,threat,obscenity
0,скотина! что сказать,0,1,0,0
1,я сегодня проезжала по рабочей и между домами ...,1,0,0,0
2,очередной лохотрон. зачем придумывать очередно...,1,0,0,0
3,"ретро дежавю ... сложно понять чужое сердце , ...",1,0,0,0
4,а когда мы статус агрогородка получили?,1,0,0,0
...,...,...,...,...,...
248285,правильно всё по пять (5)...,1,0,0,0
248286,ёбанные нубы заходите на сервер мой ник _creep...,0,1,0,0
248287,а у меня наверное рекорд в 1962 году в училище...,1,0,0,0
248288,спасибо всем большое),1,0,0,0


In [152]:
df2.normal.value_counts()

normal
1    203685
0     44605
Name: count, dtype: int64

In [153]:
df2.comment[1]

'я сегодня проезжала по рабочей и между домами снитенко и гомолысовой магазином ( на пустыре) бежала кошка похожего окраса. может, я и ошиблась, но необычный окрас бросился в глаза.'

In [154]:
# df2.to_csv("comments_250k.csv", index=False, encoding='utf-8-sig')

### Очистка данных

In [155]:
df2

,comment,normal,insult,threat,obscenity
0,скотина! что сказать,0,1,0,0
1,я сегодня проезжала по рабочей и между домами ...,1,0,0,0
2,очередной лохотрон. зачем придумывать очередно...,1,0,0,0
3,"ретро дежавю ... сложно понять чужое сердце , ...",1,0,0,0
4,а когда мы статус агрогородка получили?,1,0,0,0
...,...,...,...,...,...
248285,правильно всё по пять (5)...,1,0,0,0
248286,ёбанные нубы заходите на сервер мой ник _creep...,0,1,0,0
248287,а у меня наверное рекорд в 1962 году в училище...,1,0,0,0
248288,спасибо всем большое),1,0,0,0


In [156]:
df2.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 248290 entries, 0 to 248289
Data columns (total 5 columns):
 #   Column     Non-Null Count   Dtype 
---  ------     --------------   ----- 
 0   comment    248290 non-null  object
 1   normal     248290 non-null  int64 
 2   insult     248290 non-null  int64 
 3   threat     248290 non-null  int64 
 4   obscenity  248290 non-null  int64 
dtypes: int64(4), object(1)
memory usage: 9.5+ MB


In [157]:
# Перевод комментов в строчный вид
df2['comment'] = df2['comment'].str.lower()
# Удаление лишних пробелов по бокам и символы переноса строки
df2['comment'] = df2['comment'].str.strip()

In [158]:
df2

,comment,normal,insult,threat,obscenity
0,скотина! что сказать,0,1,0,0
1,я сегодня проезжала по рабочей и между домами ...,1,0,0,0
2,очередной лохотрон. зачем придумывать очередно...,1,0,0,0
3,"ретро дежавю ... сложно понять чужое сердце , ...",1,0,0,0
4,а когда мы статус агрогородка получили?,1,0,0,0
...,...,...,...,...,...
248285,правильно всё по пять (5)...,1,0,0,0
248286,ёбанные нубы заходите на сервер мой ник _creep...,0,1,0,0
248287,а у меня наверное рекорд в 1962 году в училище...,1,0,0,0
248288,спасибо всем большое),1,0,0,0


In [159]:
df2.comment[1]

'я сегодня проезжала по рабочей и между домами снитенко и гомолысовой магазином ( на пустыре) бежала кошка похожего окраса. может, я и ошиблась, но необычный окрас бросился в глаза.'

In [160]:
import re

def clean_text(text):
    # Удаление URL
    text = re.sub(r'https?://\S+|www\.\S+', '', text)
    # Удаление символов отлчных от ASCII
    text = re.sub('[^a-zA-Zа-яА-ЯёЁ]+', ' ', text)
    # Удаление пунктуации и спецсимволов
    text = re.sub(r'[^\w\s]', ' ', text)
    # Удаление лишних пробелов
    text = re.sub(r'\s+', ' ', text).strip()
    
    return text

In [161]:
df2['comment'] = df2['comment'].apply(clean_text)

In [162]:
df2.comment[248285]

'правильно всё по пять'

### Токенизация

In [165]:
import nltk
from pymorphy2 import MorphAnalyzer
from nltk.corpus import stopwords

In [170]:
def full_preprocess(text):
    # Токенизация
    tokens = nltk.word_tokenize(text)
    words = [t for t in tokens]
    # Удаление стоп-слов
    filtered_words = [w for w in words if w not in stop_words]
    # Лемматизация
    lemmas = [morph.parse(w)[0].normal_form for w in filtered_words]
    
    return lemmas